In [1]:

from asl.training import preprocess
from asl.utils import selected_columns,decode_batch_predictions,decode_phrase
from asl.constants import Constants
from asl.model import get_model
from asl.config import CFG

import tensorflow as tf
import json
import pandas as pd
import numpy as np


2023-07-13 12:15:51.124019: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 12:15:51.667400: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1
REPLICAS: 1


2023-07-13 12:15:52.431604: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-13 12:15:52.450550: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-13 12:15:52.450681: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
config=CFG
model = get_model(
    max_len=config.max_len,
    output_dim=config.output_dim,
    dim=config.dim,
    input_pad=Constants.INPUT_PAD,
)
experiment_id=0
saved_based_model = f"{config.log_path}/{config.comment}-exp{experiment_id}-best.h5"
model.load_weights(saved_based_model)
print(f"model with weights {saved_based_model}")

model with weights logs/model-384-seed42-exp0-best.h5


In [3]:
class InferModel(tf.Module):
    def __init__(self, model,config=CFG):
        super().__init__()

        self.model = model
        self.max_len=config.max_len

    @tf.function(
        input_signature=[tf.TensorSpec(shape=(None,Constants.NUM_INPUT_FEATURES), dtype=tf.float32, name="inputs")]
    )
    def __call__(self, inputs):
        """
        Applies the feature generation model and main model to the input tensor.

        Args:
            inputs: Input tensor with shape (T, F).

        Returns:
            A dictionary with a single key 'outputs' and corresponding output tensor.
        """
        x = preprocess(tf.cast(inputs, dtype=tf.float32),max_len=self.max_len)
        ctc = self.model(x[None,...])[0]
        decode=decode_phrase(ctc)
        outputs=tf.one_hot(decode,depth=60)
        return {"outputs": outputs}






In [4]:
# Sanity Check
with open ("/kaggle/input/asl-fingerspelling/character_to_prediction_index.json", "r") as f:
    character_map = json.load(f)
rev_character_map = {j:i for i,j in character_map.items()}

infer_keras_model=InferModel(model)

main_dir = '/kaggle/input/asl-fingerspelling/'
path = f'{main_dir}train_landmarks/5414471.parquet'
cols=selected_columns(path)
df = pd.read_parquet(path, engine = 'auto', columns = cols)
seq_id=1816796431
seq=df.loc[seq_id]
data = seq[cols].to_numpy()
print(f'input shape: {data.shape}, dtype: {data.dtype}')
output = infer_keras_model(data)["outputs"]
prediction_str = "".join([rev_character_map.get(s, "") for s in np.argmax(output, axis=1)])

#prediction_str = decode_batch_predictions(output[None,...])[0]
print(prediction_str)

input shape: (123, 244), dtype: float32


2023-07-13 12:15:57.009991: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-07-13 12:15:57.053852: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8801
2023-07-13 12:15:57.161771: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


3 creek house


In [9]:
SAVED_MODEL_PATH="/kaggle/working/infer_model"
tf.saved_model.save(infer_keras_model,SAVED_MODEL_PATH)
keras_model_converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_PATH)
keras_model_converter.optimizations = [tf.lite.Optimize.DEFAULT]
#keras_model_converter.target_spec.supported_types = [tf.float16]
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
#converter.allow_custom_ops=True
tflite_model = keras_model_converter.convert()
TFLITE_FILE_PATH="/kaggle/working/model.tflite"
with open(TFLITE_FILE_PATH, "wb") as f:
    f.write(tflite_model)

with open('/kaggle/working/inference_args.json', 'w') as f:
     json.dump({ 'selected_columns': cols }, f)

#!zip submission.zip "/kaggle/working/model.tflite" "/kaggle/working/inference_args.json"

INFO:tensorflow:Assets written to: /kaggle/working/infer_model/assets


INFO:tensorflow:Assets written to: /kaggle/working/infer_model/assets
2023-07-13 12:20:33.984557: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-07-13 12:20:33.984581: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-07-13 12:20:33.984740: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /kaggle/working/infer_model
2023-07-13 12:20:34.017102: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-07-13 12:20:34.017129: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /kaggle/working/infer_model
2023-07-13 12:20:34.104837: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-07-13 12:20:34.376141: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /kaggle/working/infer_model
2023-07-13 12:20:34.551185: I tensorf

updating: kaggle/working/model.tflite (deflated 17%)
  adding: kaggle/working/inference_args.json (deflated 83%)


In [7]:

interpreter = tf.lite.Interpreter(TFLITE_FILE_PATH)
REQUIRED_SIGNATURE = "serving_default"
REQUIRED_OUTPUT = "outputs"
found_signatures = list(interpreter.get_signature_list().keys())
if REQUIRED_SIGNATURE not in found_signatures:
    print("Required input signature not found.")

prediction_fn = interpreter.get_signature_runner("serving_default")
output = prediction_fn(inputs=data)
prediction_str = "".join([rev_character_map.get(s, "") for s in np.argmax(output[REQUIRED_OUTPUT], axis=1)])
print(prediction_str)

['serving_default']
3 creek house
